#### Summary:
- 3 baselines, copy the value from last period, copy the value from the same period the year before, take the max between the last 2 periods
- Models on geo features alone can at best match the performance of the baseline
- Need to find a way to bring some improvement by adding conflict features!!

### 1286 rows in test, with models i predict all of them

It is high because many areas are missing. options are:
- inner join and only count what is there (PPS has highest accuracy but predicts 50% less areas)
- left join and fillna(0) which keeps all rows from test but lowers accuracy ad all missing areas are considered wrong
- left join and fillna(witht the correct score) but that artificially inflates the accuracy
- Use a previous period

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import plotly.express as px
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score, multilabel_confusion_matrix
from xgboost import XGBClassifier
from sklearn import metrics, preprocessing
from sklearn.calibration import CalibratedClassifierCV
import matplotlib.pyplot as plt
import seaborn as sns
#from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.utils import class_weight

In [ ]:
df = pd.read_parquet("../data_2023/acled_data/processed/fews_with_conflicts_admin2.parquet")
#df = df[(df['ADMIN0'] == 'Burkina Faso') & (df['ADMIN2'] == 'Bale')]
df.head(25)

### Fit a classifier to predict the exact IPC score
- We start by creating a naive rule based classifier to be used as baseline
- We then train and test actual models and compare

In [ ]:
df = pd.read_parquet("../data_2023/acled_data/processed/fews_with_conflicts_admin2.parquet")
df = df[df['CS_score'].isin([1.0, 2.0, 3.0, 4.0, 5.0])]

df['CS_score'] = np.ceil(df['CS_score'])
train = df[df['period'] == '202310']
test = df[df['period'] == '202402']

train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

print(len(test.index))
print(len(train.index))

train = train[['ADMIN0', 'ADMIN1', 'ADMIN2', 'CS_score', 'period']]
train.rename(columns={'CS_score':'predicted'}, inplace=True)

test = pd.merge(test, train, on = ['ADMIN0', 'ADMIN1', 'ADMIN2'], how='left')
print(len(test.index))


In [ ]:
df = pd.read_parquet("../data_2023/acled_data/processed/fews_with_conflicts_admin2.parquet")
df = df[df['CS_score'].isin([1.0, 2.0, 3.0, 4.0, 5.0])]

df['CS_score'] = np.ceil(df['CS_score'])
train = df[df['period'] != max(df['period'])]
test = df[df['period'] == max(df['period'])]
print(max(df['period']))

base_summary = pd.DataFrame(columns = ['Model', 'Test Accuracy', 'Test Precision','Test Recall', 'F1'])
base_models = ['PPS', 'SPLY', 'Max-2PP']
accuracies = []
precisions = []
recalls = []
f1s = []
# First naive model: score is the same as the score in the last period (PPS)
train = train[train['period'].isin(sorted(train['period'].unique())[-1:])]
print(train['period'].unique())

train = train[['ADMIN0', 'ADMIN1', 'ADMIN2', 'CS_score', 'period']]
train.rename(columns={'CS_score':'predicted'}, inplace=True)
test = pd.merge(test, train, on = ['ADMIN0', 'ADMIN1', 'ADMIN2'], how='left')
test['predicted'] = test['predicted'].fillna(0)
print(len(test.index))

accuracies.append(accuracy_score(test['CS_score'], test['predicted']))
precisions.append(precision_score(test['CS_score'], test['predicted'], average='weighted'))
recalls.append(recall_score(test['CS_score'], test['predicted'], average='weighted'))
f1s.append(f1_score(test['CS_score'], test['predicted'], average='weighted'))

# Second naive model: score is the same as the score in the same period last year (SPLY)
train = df[df['period'] != max(df['period'])]
test = df[df['period'] == max(df['period'])]

train = train[train['period'].isin(sorted(train['period'].unique())[-3:-2])]
print(train['period'].unique())

train = train[['ADMIN0', 'ADMIN1' , 'ADMIN2', 'CS_score', 'period']]
train.rename(columns={'CS_score':'predicted'}, inplace=True)
test = pd.merge(test, train, on = ['ADMIN0', 'ADMIN1', 'ADMIN2'], how='left')
test['predicted'] = test['predicted'].fillna(0)
print(len(test.index))

accuracies.append(accuracy_score(test['CS_score'], test['predicted']))
precisions.append(precision_score(test['CS_score'], test['predicted'], average='weighted'))
recalls.append(recall_score(test['CS_score'], test['predicted'], average='weighted'))
f1s.append(f1_score(test['CS_score'], test['predicted'], average='weighted'))

# Third naive model: take max score of the previous 2 periods (Max-2PP)
train = df[df['period'] != max(df['period'])]
test = df[df['period'] == max(df['period'])]

train =  train[train['period'].isin(sorted(train['period'].unique())[-2:])]
print(train['period'].unique())

train = train[['ADMIN0', 'ADMIN1' , 'ADMIN2', 'CS_score', 'period']]
train = pd.DataFrame(train.groupby(['ADMIN0', 'ADMIN1', 'ADMIN2'])['CS_score'].max()).reset_index()
train.rename(columns={'CS_score':'predicted'}, inplace=True)
test = pd.merge(test, train, on = ['ADMIN0', 'ADMIN1', 'ADMIN2'], how='left')
test['predicted'] = test['predicted'].fillna(0)
print(len(test.index))

accuracies.append(accuracy_score(test['CS_score'], test['predicted']))
precisions.append(precision_score(test['CS_score'], test['predicted'], average='weighted'))
recalls.append(recall_score(test['CS_score'], test['predicted'], average='weighted'))
f1s.append(f1_score(test['CS_score'], test['predicted'], average='weighted'))

base_summary['Model'] = base_models
base_summary['Test Accuracy'] = accuracies
base_summary['Test Precision'] = precisions
base_summary['Test Recall'] = recalls
base_summary['F1'] = f1s
base_summary

## Model only on historical FS data

In [ ]:
df = pd.read_parquet("../data_2023/acled_data/processed/fews_with_conflicts_admin2.parquet")
df = df[df['CS_score'].isin([1.0, 2.0, 3.0, 4.0, 5.0])]

df['region'] = df['ADMIN0'] + '-' + df['ADMIN1'] + '-' + df['ADMIN2']
df = df.drop(['ADMIN1', 'ADMIN2'], axis=1)
df['CS_score_class'] = np.ceil(df['CS_score'])

## New features
df['transitions'] = df.groupby('region')['region'].transform(lambda x: (x != x.shift()).sum())
df['previous_CS'] = df.groupby('region')['CS_score_class'].shift(1)

df = df[df['period'].isin(sorted(df['period'].unique())[-7:])]

use_features = ['region', 'period', 'CS_score_class', 'ADMIN0', 'previous_CS', 'transitions']

df = df[use_features]
df = pd.get_dummies(df, columns=['region', 'ADMIN0'], dtype=int)

cols = list(df.columns)
cols.remove('CS_score_class')
cols.remove('period')

train = df[df['period'] != max(df['period'])]
train = train[train['previous_CS'].isin([1.0, 2.0, 3.0, 4.0, 5.0])]
train = train[train['period'].isin(sorted(train['period'].unique())[-3:])]

test = df[df['period'] == max(df['period'])]
test = test[test['previous_CS'].isin([1.0, 2.0, 3.0, 4.0, 5.0])]

X = train[cols]
y = train['CS_score_class']
X_test = test[cols]
y_test = test['CS_score_class']

LR_model = LogisticRegression(class_weight = 'balanced').fit(X, y)  
print("LR DONE")

RF_model = RandomForestClassifier(n_estimators = 50, min_samples_split=30, class_weight = 'balanced', n_jobs = -1, random_state=5).fit(X, y)
#Best Parameters: {'bootstrap': True, 'class_weight': 'balanced', 'max_depth': 35, 'min_samples_leaf': 1, 'min_samples_split': 40, 'n_estimators': 50}
print("RF DONE")

classes_weights = class_weight.compute_sample_weight(
    class_weight='balanced',
    y=train['CS_score_class']
)
GB_model = XGBClassifier(n_estimators=10, max_depth=30,  n_jobs = -1, random_state=5).fit(X, (np.array(y, dtype=int) - 1), sample_weight=classes_weights)
#print("GB DONE")

print(len(y_test))

models = [LR_model, RF_model]#, GB_model]

summary = pd.DataFrame(columns = ['Model', 
                                  #'Train Accuracy', 'Train Precision','Train Recall', 
                            'Test Accuracy', 'Test Precision','Test Recall', 'F1'])

for m in models:
    print(len(m.predict(X_test)))
    row = list()
    row.append(str(m))
    if m != GB_model:
        row.append(accuracy_score(y_test, m.predict(X_test)))
        row.append(precision_score(y_test, m.predict(X_test), average='weighted'))
        row.append(recall_score(y_test, m.predict(X_test), average='weighted'))
        row.append(f1_score(y_test, m.predict(X_test), average='weighted'))
    else:
        row.append(accuracy_score((np.array(y_test, dtype=int) - 1), m.predict(X_test)))
        row.append(precision_score((np.array(y_test, dtype=int) - 1), m.predict(X_test), average='weighted'))
        row.append(recall_score((np.array(y_test, dtype=int) - 1), m.predict(X_test), average='weighted'))
        row.append(f1_score((np.array(y_test, dtype=int) - 1), m.predict(X_test), average='weighted'))
    summary_length = len(summary)
    summary.loc[summary_length] = row

summary = pd.concat([base_summary, summary]).reset_index()
summary

In [ ]:
0.777 - 0.744

In [ ]:
# Create larger figure size
plt.figure(figsize=(12, 10))

mcm = confusion_matrix(y_test, LR_model.predict(X_test))
ax = sns.heatmap(mcm, annot=True, cmap='Blues', fmt='g', annot_kws={'size': 14})

ax.set_title('Confusion Matrix\n', fontsize=16, pad=20)
ax.set_xlabel('\nPredicted Values', fontsize=14)
ax.set_ylabel('Actual Values', fontsize=14)

# Ticket labels
ax.xaxis.set_ticklabels(['1','2', '3', '4'], fontsize=12)
ax.yaxis.set_ticklabels(['1','2', '3', '4'], fontsize=12)

plt.tight_layout()
plt.show()

In [ ]:
# Define the parameter grid for Random Forest
parameters = {
    'n_estimators': [10, 20, 30, 50, 100],
    'max_depth': [5, 10, 35,],
    'min_samples_leaf': [1, 5, 10, 40],
    'min_samples_split': [2, 10, 40],
    'bootstrap': [True, False],
    'class_weight' : ['balanced']
}

# Create the Random Forest model
rf_model = RandomForestClassifier()

# Create GridSearchCV
grid_search = GridSearchCV(rf_model, parameters, scoring='accuracy', cv=5, n_jobs=-1, verbose=10)

# Fit the model
#grid_search.fit(X, y)

# Display the best parameters
#print("Best Parameters:", grid_search.best_params_)

## Model that includes conflicts data

In [ ]:
df = pd.read_parquet("../data_2023/acled_data/processed/fews_with_conflicts_admin2.parquet")
df = df[df['CS_score'].isin([1.0, 2.0, 3.0, 4.0, 5.0])]

df['region'] = df['ADMIN0'] + '-' + df['ADMIN1'] + '-' + df['ADMIN2']
df = df.drop(['ADMIN1', 'ADMIN2'], axis=1)
df['CS_score_class'] = np.ceil(df['CS_score'])

## New features
# ADD IF CONFLICTS ARE SIGNIFICANT OR NOT FOR THE REGION BASED ON PREVIOUS WORK
df['transitions'] = df.groupby('region')['CS_score_class'].transform(lambda x: (x != x.shift()).sum())
df['total_conflicts'] = df.groupby('region')['conflicts'].transform(pd.Series.cumsum)
df['total_fatalities'] = df.groupby('region')['fatalities'].transform(pd.Series.cumsum)
df['previous_CS'] = df.groupby('region')['CS_score_class'].shift(1)

df = df[df['period'].isin(sorted(df['period'].unique())[-7:])]

use_features = ['region', 'period', 'CS_score_class', 'ADMIN0', 'previous_CS',
                'conflicts', 
                'transitions', 
                'fatalities',
                'total_conflicts', 
                'total_fatalities', 
                ]

## Scaling non binary features
min_max_scaler = preprocessing.MinMaxScaler()
df[['conflicts', 'total_conflicts', 'fatalities', 'total_fatalities']] = min_max_scaler.fit_transform(df[['conflicts', 'total_conflicts', 'fatalities', 'total_fatalities']])

df = df[use_features]
df = pd.get_dummies(df, columns=['region', 'ADMIN0'], dtype=int)

cols = list(df.columns)
cols.remove('CS_score_class')
cols.remove('period')

train = df[df['period'] != max(df['period'])]
train = train[train['previous_CS'].isin([1.0, 2.0, 3.0, 4.0, 5.0])]
train = train[train['period'].isin(sorted(train['period'].unique())[-3:])]

test = df[df['period'] == max(df['period'])]
test = test[test['previous_CS'].isin([1.0, 2.0, 3.0, 4.0, 5.0])]

X = train[cols]
y = train['CS_score_class']
X_test = test[cols]
y_test = test['CS_score_class']
oversample = ADASYN(sampling_strategy='minority')
#X, y = oversample.fit_resample(X, y)

LR_model = LogisticRegression(class_weight = 'balanced').fit(X, y)  
print("LR DONE")

RF_model = RandomForestClassifier(n_estimators = 50, min_samples_split=30, class_weight = 'balanced', n_jobs = -1, random_state=5).fit(X, y)
#Best Parameters: {'bootstrap': True, 'class_weight': 'balanced', 'max_depth': 35, 'min_samples_leaf': 1, 'min_samples_split': 40, 'n_estimators': 50}
print("RF DONE")

classes_weights = class_weight.compute_sample_weight(
    class_weight='balanced',
    y=train['CS_score_class']
)
#GB_model = XGBClassifier(n_estimators=100, max_depth=20,  n_jobs = -1, random_state=5).fit(X, (np.array(y, dtype=int) - 1), sample_weight=classes_weights)
#print("GB DONE")

print(len(y_test))

models = [LR_model, RF_model]

summary = pd.DataFrame(columns = ['Model', 
                                  #'Train Accuracy', 'Train Precision','Train Recall', 
                            'Test Accuracy', 'Test Precision','Test Recall', 'F1'])

for m in models:
    print(len(m.predict(X_test)))
    row = list()
    row.append(str(m))
    if m != GB_model:
        row.append(accuracy_score(y_test, m.predict(X_test)))
        row.append(precision_score(y_test, m.predict(X_test), average='weighted'))
        row.append(recall_score(y_test, m.predict(X_test), average='weighted'))
        row.append(f1_score(y_test, m.predict(X_test), average='weighted'))
    else:
        row.append(accuracy_score((np.array(y_test, dtype=int) - 1), m.predict(X_test)))
        row.append(precision_score((np.array(y_test, dtype=int) - 1), m.predict(X_test), average='weighted'))
        row.append(recall_score((np.array(y_test, dtype=int) - 1), m.predict(X_test), average='weighted'))
        row.append(f1_score((np.array(y_test, dtype=int) - 1), m.predict(X_test), average='weighted'))
    summary_length = len(summary)
    summary.loc[summary_length] = row

summary = pd.concat([base_summary, summary]).reset_index()
summary

In [ ]:
features = pd.DataFrame(list(zip(X.columns, RF_model.feature_importances_)), columns =['Features', 'importance'])
features = features[features['importance'] >= 0.005]
features = features.sort_values('importance', ascending=True)
fig = px.bar(features, x="importance", y="Features", orientation='h')
fig.update_layout(title_text="Feature importance",
                  title_x=0.5)
fig.show()

In [ ]:
# Create larger figure size
plt.figure(figsize=(12, 10))

mcm = confusion_matrix(y_test, LR_model.predict(X_test))
ax = sns.heatmap(mcm, annot=True, cmap='Blues', fmt='g', annot_kws={'size': 14})

ax.set_title('Confusion Matrix\n', fontsize=20, pad=20)
ax.set_xlabel('\nPredicted Values', fontsize=14)
ax.set_ylabel('Actual Values', fontsize=14)

# Ticket labels
ax.xaxis.set_ticklabels(['1','2', '3', '4'], fontsize=12)
ax.yaxis.set_ticklabels(['1','2', '3', '4'], fontsize=12)

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Compute confusion matrix
mcm = confusion_matrix(y_test, LR_model.predict(X_test))

# Convert to percentage (relative to total observations)
mcm_percent = mcm / mcm.sum()

# Create larger figure
plt.figure(figsize=(12, 10))

# Plot heatmap, fmt='.2%' displays values as percentages
ax = sns.heatmap(
    mcm_percent, 
    annot=True, 
    cmap='Blues', 
    fmt='.2%',   # Format as percentage with 2 decimals
    annot_kws={'size': 14}
)

ax.set_title('Confusion Matrix (Percentage)\n', fontsize=20, pad=20)
ax.set_xlabel('\nPredicted Values', fontsize=14)
ax.set_ylabel('Actual Values', fontsize=14)

ax.xaxis.set_ticklabels(['1','2','3','4'], fontsize=15)
ax.yaxis.set_ticklabels(['1','2','3','4'], fontsize=15)

plt.tight_layout()
plt.show()

In [ ]:
print(mcm_percent)

In [ ]:
df = pd.read_parquet("../data/acled_data/processed/fews_with_conflicts_admin2.parquet")

df['region'] = df['ADMIN0'] + '-' + df['ADMIN1'] + '-' + df['ADMIN2']
df = df.drop(['ADMIN1', 'ADMIN2'], axis=1)
df = df[df['CS_score'].notna() & df['CS_score'] != 0.0]
df['CS_score_class'] = np.ceil(df['CS_score'])

## New features
df['transitions'] = df.groupby('region')['CS_score_class'].transform(pd.Series.cumsum)
df['total_conflicts'] = df.groupby('region')['conflicts'].transform(pd.Series.cumsum)
df['total_fatalities'] = df.groupby('region')['fatalities'].transform(pd.Series.cumsum)
df['previous_CS'] = df.groupby('region')['CS_score_class'].shift(1)

use_features = ['region', 'period', 'CS_score_class', 'ADMIN0', 
                'conflicts', 'transitions', 'total_conflicts', 'total_fatalities', 'previous_CS']

df = df[use_features]
df = df[df['period'] >= '202106']
df = pd.get_dummies(df, columns=['region', 'ADMIN0'])

df.head()